# Run a distilled model from DeepSeek

The original DeepSeek model has 671B parameters and can only be run on a (big!)
cluster of H200 or GB200 GPUs, which is not suitable for us.

The distills have similar features, but are much smaller.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

This distill is based on Qwen3

In [ ]:
model_name = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"

Be sure to always load the model and the tokenizer with the same name. Otherwise, results are completely arbitrary.

Moreover, you *have* to use `dtype="auto"`. Otherwise, all data is converted to 32 bit which is slower and needs
twice the memory

In [ ]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="auto"
)

Check memory usage

In [ ]:
!nvidia-smi

Play with different prompts and `enable_thinking`. You can also modify the system prompt!

In [ ]:
# prepare the model input
# prompt = "Explain O'Reilly online learning"
prompt = "How many 'r's are in 'strawberry'?"
#prompt = "How many 'i's are in 'inscription'?"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)

The text is not yet tokenized, so you can observe the effect of the chat template.

In [ ]:
# we check what happens when we remove thinking later
text

As model inputs, we need the `id`s of the tokens.

In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

Check the speed of the generation process.

In [ ]:
%%time 
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)

The model returns the text including the prompt, so skip our own input.

In [ ]:
# only read output, skip input
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

Here we could calculate the number of tokens per second:

In [ ]:
len(output_ids)

Split the data into the thinking process and the solution.
You can get the `token_id`s from `tokenizer_config.json`.

In [ ]:
# possibly two different content parts
try:
    # rindex finding 151668 (</think>), could also look for the string!
    think_index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    think_index = 0

thinking_content = tokenizer.decode(output_ids[:think_index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[think_index:], skip_special_tokens=True).strip("\n")

These are the thoughts of the model

In [ ]:
print(thinking_content)

And this is the solution:

In [ ]:
print(content)

Most models producs results in *markdown*, which can be converted to HTML:

In [ ]:
from IPython.display import display, Markdown
display(Markdown(content))